In [18]:
diseases=['covid' , 'tuberculosis', 'asthma']
context1=['diagnosis', 'medications','types', 'facts']
context2=['sideeffects', 'preparation', 'duration']


diseasesKeywords= {
    "covid": ["covid", "covid-19", "covid19", "sars-cov"],
    "tuberculosis": ['tuberculosis', 'tb'],
    "asthma" : ["asthma"],
    "nocontext": [""]
}
context1 = {
    "diagnosis":["if i have", "if he has", "if she has", "if they have", "diagnosis"],
    "medications": ["treatment", "medications", "medicine"],
    "types":["types", "variants", "stages"],
    "facts":["what is that?","tell me", "when did", "what happens"],
    "nocontext":[""]
}

context2= {
    "sideeffects":["side effects", "careful", "effects"],
    "preparation":["preparation", "prepare for", "requirements"],
    "duration": ["how long", "duration"],
    "nocontext":[""]
}

In [33]:
def printContext(text):
    diseaseaseContext="nocontext"
    contexta="nocontext"
    contextb="nocontext"
    for key in diseasesKeywords:
        for keyword in diseasesKeywords[key]:
            if keyword in text:
                diseaseaseContext=key
                break
        else:
            continue
        break
    for key in context1:
        for keyword in context1[key]:
            if keyword in text:
                contexta=key
                break
        else:
            continue
        break
    for key in context2:
        for keyword in context2[key]:
            if keyword in text:
                contextb=key
                break
        else:
            continue
        break
    return(diseaseaseContext, contexta, contextb)

In [39]:
count=0
count= len(diseases)*len(context1)*len(context2)
print(count)
        